Парсер Авито для стат. исследований.

Данный вариант выгружает резюме по запросу.

Выгружает только данные, нужные для исследований, без телефонов и прочих контактов.

Часть 1. Сбор ссылок.

Парсер доработан 27 октября 2021.

In [1]:
#Загружаю нужные библиотеки

import pandas as pd

# регулярки
import re

# для округления в большую сторону
import math

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [3]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [4]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
#opt.add_argument('--proxy-server=91.213.35.142:53240') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [5]:
# запускаю драйвер, проверяю настройки

driver = webdriver.Chrome(options=opt)
driver.get(test_url)

In [6]:
# администратор стоматологии / Медицина, фармацевтика

url_part_1 = 'https://www.avito.ru/rossiya/rezume?bt=1&p='

resume_queries = ['администратор+стоматологии', 'администратор+стоматологической', 'администратор+клиники']

In [7]:
def parse_page():
    
    temp_data = []
    
    resume_blocks = driver.find_elements_by_class_name('iva-item-content-UnQQ4')
    
    for resume in resume_blocks:

        try: # заголовок объявления
            topic = resume.find_element_by_tag_name('h3').text

        except:
            topic = 'none'

        try: # зарплатные ожидания
            salary = resume.find_element_by_class_name('price-text-E1Y7h').text

        except:
            salary = 'none'


        try: # краткое описание
            short_description = resume.find_element_by_class_name('iva-item-descriptionStep-QGE8Y').text

        except:
            short_description = 'none'


        try: # пол, возраст, город
            parameters = resume.find_element_by_tag_name('div[data-marker="item-line"]').text

        except:
            parameters = 'none'


        try: # ссылка на страницу резюме
            ad_url = resume.find_element_by_tag_name('a').get_attribute('href')

        except:
            ad_url = 'none'
            
        temp_data.append([topic, salary, short_description, parameters, ad_url])
        
    return temp_data

In [12]:
# собираю ссылки

data = [] # список для данных

for query in tqdm(resume_queries): # цикл для перебора запросов
    
    driver.get(url_part_1 + '1' + '&q=' + query)
    sleep(3)
    
    try:
        page_number = int((driver
                           .find_element_by_class_name('pagination-root-Ntd_O')
                           .text.replace('След.', '')
                           .split()
                          )[1][-1]) + 1
    except:
        page_number = 2
        
    data = data + parse_page()
    
    
    
    if page_number > 2:
        
        for i in range(2, page_number):
            driver.get(url_part_1 + str(i) + '&q=' + query)
            sleep(3)
            
            data = data + parse_page()

  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
head_list = ['topic', 'salary', 'short_description', 'parameters', 'ad_url']
work_links = pd.DataFrame(data, columns=head_list)

In [14]:
work_links.sample()

,topic,salary,short_description,parameters,ad_url
303,"Администратор школы, салона, клиники",Зарплата не указана,none,"Женщина, 39 лет, без опыта работы, Краснообск",https://www.avito.ru/krasnoobsk/rezume/adminis...


In [15]:
work_links.shape

(467, 5)

In [17]:
# Записываю в файл

work_links.to_excel('C:/00_Data/avito_resume_2021_10_28.xlsx', encoding='utf-8', index=False)

In [18]:
# закрываю Селениум
driver.close()